In [1]:
from operator import itemgetter
from typing import List, Tuple
from chromadb import Settings
import chromadb
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    format_document,
)
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [2]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

In [1]:
qw_embedding = DashScopeEmbeddings(
    model="text-embedding-v2", dashscope_api_key=os.getenv('DASHSCOPE_API_KEY')
)

DATA_DIR = '/Users/pangmengting/Documents/workspace/python-learning/data'
CHROMA_DATA_PATH = f"{DATA_DIR}/chroma_vector_db"
CHROMA_CLIENT = chromadb.PersistentClient(
    path=CHROMA_DATA_PATH,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
)

fix_collection_name = 'yxk-know-index-3'
persist_directory = '/Users/pangmengting/Documents/workspace/python-learning/data/chroma_vector_db'
vectordb = Chroma(collection_name=fix_collection_name,
                  client=CHROMA_CLIENT,
                  embedding_function=qw_embedding)

retriever = vectordb.as_retriever()

# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

# RAG answer synthesis prompt
template = """Answer the question based only on the following context:
<context>
{context}
</context>"""
ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{question}"),
    ]
)

# Conversational Retrieval Chain
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


# User input
class ChatHistory(BaseModel):
    chat_history: List[Tuple[str, str]] = Field(..., extra={"widget": {"type": "chat"}})
    question: str


# 一个基于条件运行两个分支之一的可运行对象。
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    # 如果输入包含聊天记录，我们将其与后续问题一起压缩
    (
        # 它接受一个参数 x，并检查 x 中是否存在键 "chat_history"。如果存在，它返回 True，否则返回 False。
        # .with_config(run_name="HasChatHistoryCheck")这是一个方法调用，
        # 配置 RunnableLambda 对象的运行名称为 "HasChatHistoryCheck"。配置名称通常用于跟踪或调试任务执行。
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question 将后续问题和聊天精简为一个独立的问题 
        # .assign(...) 是一个方法，用于创建新的键值对或修改现有的键值对。
        RunnablePassthrough.assign(
            # chat_history 键的值替换为 _format_chat_history(x["chat_history"]) 的结果
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | qw_llm_openai
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    # 使用 RunnableLambda 类来包装一个函数（在本例中是 itemgetter("question")），使其成为一个可执行对象
    # 这在数据流处理或管道处理中非常有用，尤其是在需要从复杂的数据结构中提取特定信息的场景中。
    RunnableLambda(itemgetter("question")),
)
# RunnableParallel 是一个类，用于并行处理多个输入任务，并将这些任务的结果组合在一起。
# 这可以用来处理多个不同的操作，并将它们的输出以键值对的形式返回。让我们分解代码并详细解释其中的各个部分。
# 这些任务会并行执行，然后将它们的结果组合在一起。
# 提取并返回 "question"。
# 格式化并返回 "chat_history"。
# 通过一系列操作生成并返回 "context"。
# 最终，这些结果将作为并行执行的输出，返回一个包含 {"question": ..., "chat_history": ..., "context": ...} 这样的字典。
_inputs = RunnableParallel(
    {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "context": _search_query | retriever | _combine_documents,
    }
).with_types(input_type=ChatHistory)

chain = _inputs | ANSWER_PROMPT | qw_llm_openai | StrOutputParser()

NameError: name 'DashScopeEmbeddings' is not defined

In [15]:
question = "你是谁啊"
answer = chain.invoke({"question": question, "chat_history": []})

In [16]:
answer

'我是阿里云开发的语言模型，我叫通义千问。'

In [17]:
chat_history = [(question, answer)]
answer = chain.invoke(
    {
        "question": "叫什么?",
        "chat_history": chat_history,
    }
)
answer

'我叫通义千问，是由阿里云开发的大型预训练语言模型。'